In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import os
import csv
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import layers
import tensorflow as tf



def fix_gpu():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.compat.v1.InteractiveSession(config=config)


fix_gpu()

train_dir = '../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train'
test_dir = '../input/stanford-car-dataset-by-classes-folder/car_data/car_data/test'





In [ ]:
class_names = []
with open('../input/stanford-car-dataset-by-classes-folder/names.csv', newline='') as Cars:
    cvsRead = csv.reader(Cars)
    for row in cvsRead:
        class_names.append(row[0])

img_width, img_height = 224, 224
nb_train_samples = 8144
nb_validation_samples = 8041
epochs = 20
steps_per_epoch = 256
batch_size = 64
n_classes = 196
categories = 196

# Data Generator with data augmentation
train_datagen = ImageDataGenerator(
    zoom_range= 0.2,
    rotation_range = 6,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

train = train_datagen.flow_from_directory(
    train_dir,
    class_mode='categorical',
    target_size=(img_height, img_width),
    batch_size=batch_size
)

validation = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')





In [ ]:
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(224, 224, 3),
    weights="imagenet",
    classes=categories,
    pooling="avg"
)
inp = tf.keras.Input(shape=(224,224,3))
pretrained_model.trainable = False
x = tf.keras.applications.resnet50.preprocess_input(inp)
x = pretrained_model(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.6)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
output = layers.Dense(categories, activation='softmax')(x)
model = tf.keras.Model(inputs=inp, outputs=output)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics='acc')

history = model.fit(
    x=train,
    validation_data=validation,
    epochs=epochs+10,
    callbacks=[callback]
)

model.save('photo_pred')

In [ ]:
os.listdir('/kaggle/working/photo_pred')

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/photo_pred')

for l in model.layers:
   l.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics='acc')

history = model.fit(
    x=train,
    validation_data=validation,
    epochs=epochs+10
)
model.save('final_version')

In [ ]:
!tar -zcvf newversion_newlayer.tar.gz /kaggle/working

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
im = "../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train/FIAT 500 Abarth 2012/04048.jpg"
pic = image.load_img(im, target_size=(224,224))
plt.imshow(pic)
# image = image.load_img(im, target_size=(224,224))
plt.show()
img_array = image.img_to_array(pic)
img_batch = np.expand_dims(img_array, axis=0)
img_preprocessed = preprocess_input(img_batch)
car_is = model.predict(img_preprocessed)
prediction = np.argmax(car_is[0])
print(class_names[prediction])